<a href="https://colab.research.google.com/github/Shreya0302-source/SciML-Implementations/blob/main/Bg_vpinns_approach_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Define the thermal diffusion equation: -epsilon * (u_xx + u_yy) + bx * u_x + by * u_y = f(x, y)
def convection_diffusion_equation(x, y, u, u_xx, u_yy, u_x, u_y, epsilon, bx, by):
    return -epsilon * (u_xx + u_yy) + bx * u_x + by * u_y

In [ ]:
# Define the Dirichlet boundary condition for temperature
def temperature_bc(x):
    return tf.zeros_like(tf.expand_dims(x[:, 0], axis=1))

In [ ]:
# Normalize the input data
def normalize_data(data, xmin, xmax, ymin, ymax):
    data[:, 0] = (data[:, 0] - xmin) / (xmax - xmin)
    data[:, 1] = (data[:, 1] - ymin) / (ymax - ymin)
    return data

In [ ]:
def compute_derivatives(model, x, y):
    with tf.GradientTape(persistent=True) as tape1:
        tape1.watch(x)
        tape1.watch(y)
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch(x)
            tape2.watch(y)
            u = model(tf.concat([x, y], axis=1))
        u_x = tape2.gradient(u, x)
        u_y = tape2.gradient(u, y)
    u_xx = tape1.gradient(u_x, x)
    u_yy = tape1.gradient(u_y, y)
    del tape1, tape2
    return u, u_x, u_y, u_xx, u_yy

In [ ]:
# Define the forcing function for thermal diffusion
@tf.function
def convection_diffusion_forcing_function(x, y, eps):
    return (2 * eps * (-x + tf.exp(2 * (x - 1) * eps))
            + x * y**2 + 6 * x * y
            - x * tf.exp(3 * (y - 1) * eps)
            - y**2 * tf.exp(2 * (x - 1) * eps)
            + 2 * y**2 - 6 * y * tf.exp(2 * (x - 1) * eps)
            - 2 * tf.exp(3 * (y - 1) * eps)
            + tf.exp(2*x + 3*y - 5*eps))

In [ ]:
# Define the differential equation: u_xx(x, y) + u_yy(x, y) + u(x, y) = 0
def differential_equation(x, y, u, u_xx, u_yy):
    return u_xx + u_yy + u

In [ ]:
# Define the neural network model
class VPINN(tf.keras.Model):
    def __init__(self):
        super(VPINN, self).__init__()
        self.input_layer = tf.keras.layers.InputLayer(shape=(2,))
        self.hidden_layer_1 = tf.keras.layers.Dense(
            50,  # Increased number of neurons
            activation=tf.nn.tanh,
            kernel_regularizer=tf.keras.regularizers.l2(0.01),
        )
        self.batch_norm_1 = tf.keras.layers.BatchNormalization()
        self.hidden_layer_2 = tf.keras.layers.Dense(
            50,  # Increased number of neurons
            activation=tf.nn.tanh,
            kernel_regularizer=tf.keras.regularizers.l2(0.01),
        )
        self.batch_norm_2 = tf.keras.layers.BatchNormalization()
        self.hidden_layer_3 = tf.keras.layers.Dense(
            50,  # Increased number of neurons
            activation=tf.nn.tanh,
            kernel_regularizer=tf.keras.regularizers.l2(0.01),
        )
        self.batch_norm_3 = tf.keras.layers.BatchNormalization()
        self.hidden_layer_4 = tf.keras.layers.Dense(
            50,  # Added an additional hidden layer
            activation=tf.nn.tanh,
            kernel_regularizer=tf.keras.regularizers.l2(0.01),
        )
        self.batch_norm_4 = tf.keras.layers.BatchNormalization()
        self.output_layer = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.hidden_layer_1(inputs)
        x = self.batch_norm_1(x)
        x = self.hidden_layer_2(x)
        x = self.batch_norm_2(x)
        x = self.hidden_layer_3(x)
        x = self.batch_norm_3(x)
        x = self.hidden_layer_4(x)
        x = self.batch_norm_4(x)
        return self.output_layer(x)

In [ ]:
# Define the PDE convection loss function with RMSE
def pde_loss(model, train_interior, epsilon, bx, by):
    x = tf.expand_dims(train_interior[:, 0], axis=1)
    y = tf.expand_dims(train_interior[:, 1], axis=1)

    u, u_x, u_y, u_xx, u_yy = compute_derivatives(model, x, y)
    residual = convection_diffusion_equation(x, y, u, u_xx, u_yy, u_x, u_y, epsilon, bx, by) - convection_diffusion_forcing_function(x, y, epsilon)
    return tf.sqrt(tf.reduce_mean(tf.square(residual))) # RMSE between the residuals


In [ ]:
def bc_loss(model, x_bd):
    u_pred = model(x_bd)
    u_exact = temperature_bc(x_bd)

    # Calculate RMSE loss
    # RMSE between predicted and exact boundary values
    return tf.sqrt(tf.reduce_mean(tf.square(u_pred - u_exact)))

In [ ]:
# Generate boundary points
def generate_boundary_points(xmin, xmax, ymin, ymax, num_points):
    np.random.seed(42)  # For reproducibility
    num_per_edge = num_points // 4
    remainder = num_points % 4  # Handle remainder properly

    bottom = np.column_stack((np.random.uniform(xmin, xmax, num_per_edge + (remainder > 0)), np.full((num_per_edge + (remainder > 0),), ymin)))
    right = np.column_stack((np.full((num_per_edge + (remainder > 1),), xmax), np.random.uniform(ymin, ymax, num_per_edge + (remainder > 1))))
    top = np.column_stack((np.random.uniform(xmin, xmax, num_per_edge + (remainder > 2)), np.full((num_per_edge + (remainder > 2),), ymax)))
    left = np.column_stack((np.full((num_per_edge,), xmin), np.random.uniform(ymin, ymax, num_per_edge)))

    boundary_points = np.vstack((bottom, right, top, left))

    return tf.convert_to_tensor(boundary_points, dtype=tf.float32)

In [ ]:
def generate_interior_points(xmin, xmax, ymin, ymax, num_points):
    """
    Generate random interior points inside the domain (excluding boundaries).

    Args:
        xmin, xmax: Domain limits in x-direction
        ymin, ymax: Domain limits in y-direction
        num_points: Number of points to generate

    Returns:
        tf.Tensor: Interior points as a (num_points, 2) tensor
    """
    np.random.seed(42)  # Ensures reproducibility
    delta = 1e-5  # Small shift to prevent boundary points

    x_interior = np.random.uniform(xmin + delta, xmax - delta, (num_points, 1))
    y_interior = np.random.uniform(ymin + delta, ymax - delta, (num_points, 1))

    interior_points = np.column_stack((x_interior, y_interior))

    return tf.convert_to_tensor(interior_points, dtype=tf.float32)

In [ ]:
# Generate training data
xmin, xmax, ymin, ymax = 0, 2 * np.pi, 0, 2 * np.pi
num_boundary_points = 4000
num_interior_points = 20000

In [ ]:
# Generate boundary and interior points
boundary_points = generate_boundary_points(xmin, xmax, ymin, ymax, num_boundary_points)
interior_points = generate_interior_points(xmin, xmax, ymin, ymax, num_interior_points)

# Normalize the training data
boundary_points = normalize_data(boundary_points.numpy(), xmin, xmax, ymin, ymax)
interior_points = normalize_data(interior_points.numpy(), xmin, xmax, ymin, ymax)

# Convert training data to TensorFlow tensors
boundary_points = tf.convert_to_tensor(boundary_points, dtype=tf.float32)
interior_points = tf.convert_to_tensor(interior_points, dtype=tf.float32)

In [ ]:
# Define the training step function
# @tf.function
# Adaptive training step with gradient clipping
def train_step(model, optimizer, train_interior, train_boundary, epsilon, bx, by):
    try:
        with tf.GradientTape() as tape:
            loss_pde = pde_loss(model, train_interior, epsilon, bx, by)
            loss_bc = bc_loss(model, train_boundary)
            total_loss = loss_pde + loss_bc
        grads = tape.gradient(total_loss, model.trainable_variables)
        grads, _ = tf.clip_by_global_norm(grads, 1.0)  # Gradient clipping for stability
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        return total_loss, loss_pde, loss_bc

    except Exception as e:
        print(f"Error in training step at epoch {epoch}: {str(e)}")
        return None, None, None  # Return None in case of an error

In [ ]:
def adjust_weights(alpha, beta, loss_pde, loss_bc, factor=1.05, max_value=10.0, min_value=0.1):
    if loss_pde > loss_bc:
        new_alpha = min(max(alpha / factor, min_value), max_value)
        new_beta = min(max(beta * factor, min_value), max_value)
    else:
        new_alpha = min(max(alpha * factor, min_value), max_value)
        new_beta = min(max(beta / factor, min_value), max_value)
    return new_alpha, new_beta

In [ ]:
# SSBroyden optimizer implementation
class SSBroydenOptimizer(tf.keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.1, name="SSBroyden", **kwargs):
        super(SSBroydenOptimizer, self).__init__(name, **kwargs)
        self.learning_rate = learning_rate

    def _resource_apply_dense(self, grad, var, apply_state=None):
        var.assign_sub(self.learning_rate * grad)

    def _resource_apply_sparse(self, grad, var, indices, apply_state=None):
        var.scatter_sub(tf.IndexedSlices(self.learning_rate * grad, indices))

In [ ]:
# Initialize the model and optimizer with a learning rate scheduler
model = VPINN()
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.9, staircase=True)
optimizer_1 = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
optimizer_2 = SSBroydenOptimizer(learning_rate=lr_schedule)

In [ ]:
# Initialize arrays to store loss values
total_loss_ar = []
pde_loss_ar = []
bound_loss_ar = []

# Initialize bx and by
bx = 2.0
by = 3.0

# Convert training data to TensorFlow tensors
train_boundary = tf.convert_to_tensor(boundary_points, dtype=tf.float32)
train_interior = tf.convert_to_tensor(interior_points, dtype=tf.float32)

# Initialize eps
epsilon = 0.0001

# Training loop with early stopping
epochs = 20000
patience = 1000  # Number of epochs to wait for improvement
best_loss = float('inf')
epochs_without_improvement = 0

In [ ]:
for epoch in range(epochs):
  if epoch <= 2000:
    optimizer = optimizer_1
  else:
    optimizer = optimizer_2
    # Perform a training step
    total_loss, loss_pde, loss_bc = train_step(model, optimizer, interior_points, boundary_points, epsilon, bx, by)

    # Store loss values
    total_loss_ar.append(total_loss.numpy())
    pde_loss_ar.append(loss_pde.numpy())
    bound_loss_ar.append(loss_bc.numpy())

    # Check for NaN values
    if tf.math.is_nan(total_loss):
        print(f'NaN encountered at epoch {epoch}')
        break

    # Check for improvement
    if total_loss < best_loss:
        best_loss = total_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    # Print progress
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, PDE Loss: {loss_pde.numpy()}, BC Loss: {loss_bc.numpy()}, Total Loss: {total_loss.numpy()}')

    # # Early stopping
    # if epochs_without_improvement >= patience:
    #     print(f'Stopping early at epoch {epoch}')
    #     break

In [ ]:
# Plotting the total loss, PDE loss, and boundary loss
plt.figure()
plt.yscale('log')
plt.plot(total_loss_ar, label='Total Loss')
plt.plot(pde_loss_ar, label='PDE Loss')
plt.plot(bound_loss_ar, label='Boundary Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Predict and print results
test_data = pd.read_csv(r'/kaggle/input/ziq-sciml-challenge/test.csv')
x_test = tf.convert_to_tensor(test_data['x'].values.reshape(-1, 1).astype(np.float32))
y_test = tf.convert_to_tensor(test_data['y'].values.reshape(-1, 1).astype(np.float32))

# Concatenate test inputs and make predictions
inputs_test = tf.concat([x_test, y_test], axis=1)
u_pred = model(inputs_test)

# Convert predictions to a NumPy array
u_pred_np = u_pred.numpy()

In [ ]:
# Create a DataFrame with the predictions
submission_df = pd.DataFrame({
    'ID': test_data['ID'],
    'x': test_data['x'],
    'y': test_data['y'],
    'u': u_pred_np.flatten()
})
# Save the DataFrame to a CSV file
submission_df.to_csv('/kaggle/working/submission.csv', index=False)